# Experiments with the Held and Suarez (1994) dry dynamical core

#### Notebook written by Elizabeth Maroon and Alice DuVivier

### Scientific Goals
- Understand the physical simplifications in the Held and Suarez model
- Explore how zonal mean temperature and wind cross sections change with simple parameter changes and/or polar/tropical amplification heating
- Discuss strengths/limitations of idealized atmospheric physics

### Technical Goals
- Make minor parameter changes to the implementation of Held and Suarez (1994) in the CESM Simple Models
- Load climate model output into a jupyter notebook
- Practice adjusting plot parameters in Python
- Learn how to save figures in Python

There are two possible activities that you can explore using this notebook: 

1. Polar heating anomalies (Butler et al. 2010, McGraw and Barnes 2015)
2. Exploring the parameters that set the wind damping and equilibrium profile 

First, you will load in your model's output and make a few plots to explore the spinup and assess if one year of output is sufficient or not to explore the changes to the climate.  <br>

"Pre-baked" simulations for some of the possible experiments have been run for 10 years. Your goal is to (at least partly) understand the mean climate response.  <br>

A control simulation has already been run for comparison for ~20 years (no seasonal cycle implemented). 

### Loading packages and opening netCDFs

In [ ]:
#import libraries
import numpy as np                         #standard library for numeric manipulation
import matplotlib.pyplot as plt            #plotting library
import xarray as xr                        #slick netCDF manipulation
import glob                                #for finding files via patterns a la Unix "glob"
from matplotlib.gridspec import GridSpec   #for multipanel plotting
from matplotlib import animation, rc       #a couple of plotting things for embedded animation below
from IPython.display import HTML           #to get the animation to show up in the jupyter notebook
import sys                                 #to access command line with ! 

%matplotlib notebook
#magic for plots to show up below cells

First, we load in the control climatology:

In [ ]:
#load in climatology files for control

ds_control=xr.open_dataset('/glade/scratch/emaroon/archive/control/atm/hist/control.cam.climo_yr5-18.nc')

#Ignore the warning that pops up - it's a consequence of the time in our model simulation being outside those that
#the developers of the "pandas" package deemed relevant (e.g., year 0). Pandas is a dependency of xarray. The 
#xarray developers are fixing this issue.

Next, you will read the one year of output that you just created using the xarray function "open_mfdataset()". mf stands for multi-file.  You will need to provide the path to the output.

In [ ]:
#read in your experiment
username='YOURUSERNAME'  ##FILL IN WITH YOUR OWN USERNAME
expname='YOUREXPERIMENTNAME'  ##FILL IN WITH YOUR EXPERIMENT NAME

path_to_your_exp='/glade/scratch/'+username+'/archive/'+expname+'/atm/hist/'
print('Is this the correct path?:',path_to_your_exp,'\n')

#make an ordered list of the monthly output files
list_of_files=sorted(glob.glob(path_to_your_exp+'*.h0.0001*.nc'))
print('Are these the right files?',list_of_files,'\n')

#open the Dataset from these 
ds_exp=xr.open_mfdataset(list_of_files)

Again, let's take a look at what's in the DataSet (analogous to ncdump -h filename.nc)

In [ ]:
print(ds_exp)

We're going to plot the time and zonal mean T, U, V from control and experiment simulations side-by-side. You may want to adjust the contour intervals or colormaps. The locations to change these parameters are marked with ALL CAPS. 

In [ ]:
#plot up the control zonal mean U,V,T

#load in the coordinates
lev=ds_control['lev']
lat=ds_control['lat']

#load in the variables from control simulation
T_con=ds_control['T']
U_con=ds_control['U']
V_con=ds_control['V']

#load in variables from experiment simulation
T_exp=ds_exp['T']
U_exp=ds_exp['U']
V_exp=ds_exp['V']


#define the contour levels for temperature 
min_tlev=180     #YOU MAY WANT TO ADJUST THIS MIN CONTOUR LEVEL FOR TEMPERATURE
max_tlev=316     #YOU MAY WANT TO ADJUST THIS MAX CONTOUR LEVEL FOR TEMPERATURE
delt_tlev=10      #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR TEMPERATURE
tlevs=np.arange(min_tlev,max_tlev,delt_tlev)

#define the contour levels for zonal wind
max_ulev=30             #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR ZONAL WIND
delt_ulev=max_ulev/10   #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR ZONAL WIND
ulevs=np.concatenate((np.arange(-1*max_ulev,0,delt_ulev),np.arange(delt_ulev,max_ulev+delt_ulev/10,delt_ulev)))

#define the contour levels for meridional wind
max_vlev=3.0              #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIDIONAL WIND
delt_vlev=max_vlev/10     #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIIDIONAL WIND
vlevs=np.concatenate((np.arange(-1*max_vlev,0,delt_vlev),np.arange(delt_vlev,max_vlev+delt_vlev/10,delt_vlev)))

#define colormaps for temperature and zonal wind
cmap_t=plt.cm.viridis  
cmap_u=plt.cm.RdBu_r
#YOU MAY WANT TO CHANGE UP THESE COLORMAPS. SEE AVAILABLE MATPLOTLIB OPTIONS HERE:
#https://matplotlib.org/examples/color/colormaps_reference.html
#Helpful trick: You can reverse any colormap's direction by adding "_r" to the end

#define the figure and multipanels
f=plt.figure(figsize=(10,6))
gs=GridSpec(2,2)

#plots control temperature
fig=plt.subplot(gs[0,0])
cs=plt.contourf(lat,lev,T_con.mean('lon').mean('time'),tlevs,cmap=cmap_t,extend='both')
#Note what is being plotted in contourf: We've taken T_con and taken the time and lon mean inside the plotting call.
#Being able to do this all in one line is part of what makes xarray cool.
plt.gca().invert_yaxis()  #flips the y-axis for pressure decreasing with height
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs)
cb.set_label('temperature (K)')
plt.title('control')

#plots control U (shading) and V (contours)
fig=plt.subplot(gs[1,0])
cs=plt.contourf(lat,lev,U_con.mean('lon').mean('time'),ulevs,cmap=cmap_u,extend='both')
plt.contour(lat,lev,V_con.mean('lon').mean('time'),vlevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')


#plots exp temperature
fig=plt.subplot(gs[0,1])
cs=plt.contourf(lat,lev,T_exp.mean('lon').mean('time'),tlevs,cmap=cmap_t,extend='both')
plt.gca().invert_yaxis()  
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs)
cb.set_label('temperature (K)')
plt.title('experiment')

#plots exp U (shading) and V (contours)
fig=plt.subplot(gs[1,1])
cs=plt.contourf(lat,lev,U_exp.mean('lon').mean('time'),ulevs,cmap=cmap_u,extend='both')
plt.contour(lat,lev,V_exp.mean('lon').mean('time'),vlevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')

plt.tight_layout()  #makes panels play nicely together

#If you want to save the figure, uncomment these lines
#plt.savefig('myfigure.png')

**Group Discussion Question:** Based on your knowledge of atmospheric general circulation, how much do these structures look like those in the real atmosphere?  

- HINT: Check out some of the annual-mean climatologies available here: https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-003-atmosphere-ocean-and-climate-dynamics-fall-2008/study-materials/

Next we're going to make a difference plot to see what's changed.  You may want to adjust some of the plotting contour levels. To do this, check out the lines in ALL CAPS:

In [ ]:
#plot up the difference of control minus experiment zonal mean U,V,T

#define the contour levels for temperature 
max_tlev=15                #YOU MAY WANT TO ADJUST THIS MIN CONTOUR LEVEL FOR TEMPERATURE
delt_tlev=max_tlev/10      #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR TEMPERATURE
tlevs_diff=np.concatenate((np.arange(-1*max_tlev,0,delt_tlev),np.arange(delt_tlev,max_tlev+delt_tlev/10,delt_tlev)))

#define the contour levels for zonal wind
max_ulev=10             #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR ZONAL WIND
delt_ulev=max_ulev/10   #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR ZONAL WIND
ulevs_diff=np.concatenate((np.arange(-1*max_ulev,0,delt_ulev),np.arange(delt_ulev,max_ulev+delt_ulev/10,delt_ulev)))

#define the contour levels for meridional wind
max_vlev=1.0              #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIDIONAL WIND
delt_vlev=max_vlev/10     #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIIDIONAL WIND
vlevs_diff=np.concatenate((np.arange(-1*max_vlev,0,delt_vlev),np.arange(delt_vlev,max_vlev+delt_vlev/10,delt_vlev)))

#define colormaps for temperature and zonal wind
cmap_td=plt.cm.RdBu_r 
cmap_ud=plt.cm.RdBu_r
#YOU MAY WANT TO CHANGE UP THESE COLORMAPS. SEE AVAILABLE MATPLOTLIB OPTIONS HERE:
#https://matplotlib.org/examples/color/colormaps_reference.html
#Helpful trick: You can reverse any colormap's direction by adding "_r" to the end

#define the figure and multipanels
f=plt.figure(figsize=(5,6))
gs=GridSpec(2,1)

#plots temperature difference
fig=plt.subplot(gs[0,0])
cs=plt.contourf(lat,lev,T_exp.mean('lon').mean('time')-T_con.mean('lon').mean('time'),tlevs_diff,cmap=cmap_td,extend='both')
plt.gca().invert_yaxis()  #flips the y-axis for pressure decreasing with height
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_tlev,max_tlev+delt_tlev,delt_tlev*2))
cb.set_label('temperature (K)')
plt.title('experiment minus control')

#plots U (shading) and V (contours) difference
fig=plt.subplot(gs[1,0])
cs=plt.contourf(lat,lev,U_exp.mean('lon').mean('time')-U_con.mean('lon').mean('time'),ulevs_diff,cmap=cmap_ud,extend='both')
plt.contour(lat,lev,V_exp.mean('lon').mean('time')-V_con.mean('lon').mean('time'),vlevs_diff,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')


plt.tight_layout()  #makes panels play nicely together

**Group discussion question**: Do you think the simulation has spun up sufficiently? Do we have enough output to clearly see the response to the parameter change or heating?  

To assess this question, we're going to plot a quick animation of the experiment surface temperature for the first year. The first cell below prepares the animation, the following cell displays it. <br>

To change the variable, check out the "plotthis" line below.

In [ ]:
%%capture
#
#to get this embedded animation, adapted the instructions from here: 
#https://stackoverflow.com/questions/23070305/how-can-i-make-an-animation-with-contourf?rq=1


#for bug fixes
import types


#set up the figure
fig=plt.figure();
ax=plt.gca()

ims = []
for i in range(len(ds_exp['time'])):
    plotthis=ds_exp['T'].mean('lon').isel(time=i).values[:,::-1] #CHANGE UP T->U or ->V to watch other variables spinup
    levs=tlevs    #CHANGE TO ulevs or vlevs if you change from T -> U or V
    im = plt.contourf(lat.values,lev.values[::-1],plotthis,levs,extend='both');
    plt.yticks(np.arange(200,801,200),np.arange(800,199,-200));
    plt.xlabel('latitude')
    plt.ylabel('pressure (HPa)')
    title=plt.text(0.5,1.05,'Y/M/D: '+str(ds_exp['time'].isel(time=i).values), \
                   transform=ax.transAxes,zorder=10,ha="center")
    
    ## Bug fixes for Quad Contour Set not having attributes 'set_visible', 'set_animate', or 'get_figure'
    def setvisible(self,vis):
        for c in self.collections: c.set_visible(vis)
    def setanimated(self,vis):
        for c in self.collections: c.set_animated(vis)
    def getfig(self):
        return self.figure;
    im.set_visible = types.MethodType(setvisible,im)
    im.set_animated = types.MethodType(setanimated,im)
    im.get_figure = types.MethodType(getfig,im)
    
    im.figure=fig;
    ims.append([im,title])



# call the animator. blit=True means only re-draw the parts that have changed.
anim=animation.ArtistAnimation(fig, ims, interval=500, blit=False,repeat_delay=1000)

HTML(anim.to_jshtml());

plt.rcParams["animation.html"] = "jshtml"



In [ ]:
#starts the animation

anim

Does it appear like the temperature state is still adjusting?  By the end of the first year could this just be some month-month variability?<br>

Next, let's plot a quick time series of global mean bottom sigma-layer temperature evolution:

In [ ]:
#calculates bottom sigma level global mean


plotthis=(ds_exp['T'].isel(lev=-1).mean('lon')*np.cos(lat*np.pi/180)).sum('lat')/(np.cos(lat*np.pi/180).sum('lat'))

#plots 
plt.plot(range(1,13),plotthis)
plt.xlabel('month since start')
plt.ylabel('global mean temperature')
plt.xticks(range(1,13));

### Examining already run Held Suarez simulations

There certainly are not enough months to construct a reliable climatology, so we've already run a couple of simulations for comparison. There are 10 years of output of the following experiments:


Activity 1:
- 2x the default free troposphere temperature damping (doubleKa)
- 0.5x the default free troposphere temperature damping (halfKa)
- 2x the default boundary layer wind damping (doubleKf)
- 0.5x the default boundary layer wind damping (halfKf)
- 2x the vertical stratification (doubleTz)
- 0.5x the vertical stratification (halfTz)
- 1.5x the meridional temperature gradient (largerTy)
- 1.5x the meridional temperature gradient (halfTy)

Activity 2: 
- Adding polar amplification heating (polarheat2)

Select the experiment most similar to the one that you ran. The following cell loads in the climatology (averaged over years 3-9). 

In [ ]:
#IN THE BELOW LINE, WRITE THE NAME OF THE SIMULATION TO EXAMINE:

expname='largerTy'  ###CHANGE HERE!!!

#POSSIBLE EXPERIMENTS:
#allexps=['doubleKa','doubleKf','doubleTz','halfKa','halfKf','halfTy','halfTz','largerTy','polarheat2']

expdir='/glade/scratch/emaroon/archive/'

#loads one file
ds_exp_long=xr.open_dataset(expdir+expname+'/atm/hist/'+expname+'.cam.climo.yr3-9.nc');


In [ ]:
#take a look at what output is available in the netcdf:
print(ds_exp_long)

Plot up the T, U, V fields

In [ ]:
#plot up the control zonal mean U,V,T

#load in the coordinates
lev=ds_control['lev']
lat=ds_control['lat']

#load in the variables from control simulation
T_con=ds_control['T']
U_con=ds_control['U']
V_con=ds_control['V']

#load in variables from experiment simulation
T_exp=ds_exp_long['T']
U_exp=ds_exp_long['U']
V_exp=ds_exp_long['V']


#define the contour levels for temperature 
min_tlev=180     #YOU MAY WANT TO ADJUST THIS MIN CONTOUR LEVEL FOR TEMPERATURE
max_tlev=316     #YOU MAY WANT TO ADJUST THIS MAX CONTOUR LEVEL FOR TEMPERATURE
delt_tlev=10      #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR TEMPERATURE
tlevs=np.arange(min_tlev,max_tlev,delt_tlev)

#define the contour levels for zonal wind
max_ulev=30             #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR ZONAL WIND
delt_ulev=max_ulev/10   #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR ZONAL WIND
ulevs=np.concatenate((np.arange(-1*max_ulev,0,delt_ulev),np.arange(delt_ulev,max_ulev+delt_ulev/10,delt_ulev)))

#define the contour levels for meridional wind
max_vlev=3.0              #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIDIONAL WIND
delt_vlev=max_vlev/10     #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIIDIONAL WIND
vlevs=np.concatenate((np.arange(-1*max_vlev,0,delt_vlev),np.arange(delt_vlev,max_vlev+delt_vlev/10,delt_vlev)))

#define colormaps for temperature and zonal wind
cmap_t=plt.cm.viridis  
cmap_u=plt.cm.RdBu_r
#YOU MAY WANT TO CHANGE UP THESE COLORMAPS. SEE AVAILABLE MATPLOTLIB OPTIONS HERE:
#https://matplotlib.org/examples/color/colormaps_reference.html
#Helpful trick: You can reverse any colormap's direction by adding "_r" to the end

#define the figure and multipanels
f=plt.figure(figsize=(10,6))
gs=GridSpec(2,2)

#plots control temperature
fig=plt.subplot(gs[0,0])
cs=plt.contourf(lat,lev,T_con.mean('lon').mean('time'),tlevs,cmap=cmap_t,extend='both')
#Note what is being plotted in contourf: We've taken T_con and taken the time and lon mean inside the plotting call.
#Being able to do this all in one line is part of what makes xarray cool.
plt.gca().invert_yaxis()  #flips the y-axis for pressure decreasing with height
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs)
cb.set_label('temperature (K)')
plt.title('control')

#plots control U (shading) and V (contours)
fig=plt.subplot(gs[1,0])
cs=plt.contourf(lat,lev,U_con.mean('lon').mean('time'),ulevs,cmap=cmap_u,extend='both')
plt.contour(lat,lev,V_con.mean('lon').mean('time'),vlevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')


#plots exp temperature
fig=plt.subplot(gs[0,1])
cs=plt.contourf(lat,lev,T_exp.mean('lon').mean('time'),tlevs,cmap=cmap_t,extend='both')
plt.gca().invert_yaxis()  
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs)
cb.set_label('temperature (K)')
plt.title('experiment')

#plots exp U (shading) and V (contours)
fig=plt.subplot(gs[1,1])
cs=plt.contourf(lat,lev,U_exp.mean('lon').mean('time'),ulevs,cmap=cmap_u,extend='both')
plt.contour(lat,lev,V_exp.mean('lon').mean('time'),vlevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')

plt.tight_layout()  #makes panels play nicely together

Plot the difference of the two climatologies:
    

In [ ]:
#plot up the difference of control minus experiment zonal mean U,V,T

#define the contour levels for temperature 
max_tlev=10               #YOU MAY WANT TO ADJUST THIS MIN CONTOUR LEVEL FOR TEMPERATURE
delt_tlev=max_tlev/10     #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR TEMPERATURE
tlevs_diff=np.concatenate((np.arange(-1*max_tlev,0,delt_tlev),np.arange(delt_tlev,max_tlev+delt_tlev/10,delt_tlev)))

#define the contour levels for zonal wind
max_ulev=15             #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR ZONAL WIND
delt_ulev=max_ulev/10   #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR ZONAL WIND
ulevs_diff=np.concatenate((np.arange(-1*max_ulev,0,delt_ulev),np.arange(delt_ulev,max_ulev+delt_ulev/10,delt_ulev)))

#define the contour levels for meridional wind
max_vlev=1.0              #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIDIONAL WIND
delt_vlev=max_vlev/10     #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIIDIONAL WIND
vlevs_diff=np.concatenate((np.arange(-1*max_vlev,0,delt_vlev),np.arange(delt_vlev,max_vlev+delt_vlev/10,delt_vlev)))

#define colormaps for temperature and zonal wind
cmap_td=plt.cm.RdBu_r 
cmap_ud=plt.cm.RdBu_r
#YOU MAY WANT TO CHANGE UP THESE COLORMAPS. SEE AVAILABLE MATPLOTLIB OPTIONS HERE:
#https://matplotlib.org/examples/color/colormaps_reference.html
#Helpful trick: You can reverse any colormap's direction by adding "_r" to the end

#define the figure and multipanels
f=plt.figure(figsize=(5,6))
gs=GridSpec(2,1)

#plots temperature difference
fig=plt.subplot(gs[0,0])
cs=plt.contourf(lat,lev,T_exp.mean('lon').mean('time')-T_con.mean('lon').mean('time'),tlevs_diff,cmap=cmap_td,extend='both')
plt.gca().invert_yaxis()  #flips the y-axis for pressure decreasing with height
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_tlev,max_tlev+delt_tlev,delt_tlev*2))
cb.set_label('temperature (K)')
plt.title('experiment minus control')

#plots U (shading) and V (contours) difference
fig=plt.subplot(gs[1,0])
cs=plt.contourf(lat,lev,U_exp.mean('lon').mean('time')-U_con.mean('lon').mean('time'),ulevs_diff,cmap=cmap_ud,extend='both')
plt.contour(lat,lev,V_exp.mean('lon').mean('time')-V_con.mean('lon').mean('time'),vlevs_diff,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')


plt.tight_layout()  #makes panels play nicely together

It might be useful to examine T and U with their control values as contours overlaying the change in T/U 

In [ ]:
#plot up the difference of control minus experiment zonal mean T,U with control T U contouors

#define the contour levels for temperature 
max_tlev=5               #YOU MAY WANT TO ADJUST THIS MIN CONTOUR LEVEL FOR TEMPERATURE
delt_tlev=max_tlev/10     #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR TEMPERATURE
tlevs_diff=np.concatenate((np.arange(-1*max_tlev,0,delt_tlev),np.arange(delt_tlev,max_tlev+delt_tlev/10,delt_tlev)))

#define the contour levels for zonal wind
max_ulev=15             #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR ZONAL WIND
delt_ulev=max_ulev/10   #YOU MAY WANT TO ADJUST THIS DELTA CONTOUR LEVEL FOR ZONAL WIND
ulevs_diff=np.concatenate((np.arange(-1*max_ulev,0,delt_ulev),np.arange(delt_ulev,max_ulev+delt_ulev/10,delt_ulev)))

#define the contour levels for meridional wind
max_vlev=1.0              #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIDIONAL WIND
delt_vlev=max_vlev/10     #YOU MAY WANT TO ADJUST THIS MAX/MIN CONTOUR LEVEL FOR MERIIDIONAL WIND
vlevs_diff=np.concatenate((np.arange(-1*max_vlev,0,delt_vlev),np.arange(delt_vlev,max_vlev+delt_vlev/10,delt_vlev)))

#define colormaps for temperature and zonal wind
cmap_td=plt.cm.RdBu_r 
cmap_ud=plt.cm.RdBu_r
#YOU MAY WANT TO CHANGE UP THESE COLORMAPS. SEE AVAILABLE MATPLOTLIB OPTIONS HERE:
#https://matplotlib.org/examples/color/colormaps_reference.html
#Helpful trick: You can reverse any colormap's direction by adding "_r" to the end

#define the figure and multipanels
f=plt.figure(figsize=(5,6))
gs=GridSpec(2,1)

#plots temperature difference
fig=plt.subplot(gs[0,0])
cs=plt.contourf(lat,lev,T_exp.mean('lon').mean('time')-T_con.mean('lon').mean('time'),tlevs_diff,cmap=cmap_td,extend='both')
plt.contour(lat,lev,T_con.mean('lon').mean('time'),tlevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()  #flips the y-axis for pressure decreasing with height
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_tlev,max_tlev+delt_tlev,delt_tlev*2))
cb.set_label('temperature (K)')
plt.title('experiment minus control with control T/U contours')

#plots U (shading) and V (contours) difference
fig=plt.subplot(gs[1,0])
cs=plt.contourf(lat,lev,U_exp.mean('lon').mean('time')-U_con.mean('lon').mean('time'),ulevs_diff,cmap=cmap_ud,extend='both')
plt.contour(lat,lev,U_con.mean('lon').mean('time'),ulevs,colors='black',alpha=0.4)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)')
plt.xticks(np.arange(-60,61,30))
cb=plt.colorbar(cs,spacing='proportional',ticks=np.arange(-1*max_ulev,max_ulev+delt_ulev,delt_ulev*2))
cb.set_label('zonal wind (m/s)')


plt.tight_layout()  #makes panels play nicely together

It might be useful to compare against the default equilibrium temperature profile:

In [ ]:

#default params
sigmab=0.7
ka=1/40
ks=1/4
Ty=60
Tz=10
kappa=2./7
ps=1000

teq=np.empty((len(lev),len(lat)))

for ll in range(len(lev)):
    teq[ll,:]=(315-Ty*np.sin(lat.values*np.pi/180)**2-Tz*np.log(lev.isel(lev=ll).values/ps)*\
         np.cos(lat.values*np.pi/180)**2)*((lev.isel(lev=ll).values/ps)**kappa)
    
teq[teq<200]=200


#plot the default equilibrium temperature profile

plt.contourf(lat,lev,teq,tlevs,extend='both')
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)');
cb=plt.colorbar()
cb.set_label('temperature (K)')


If you are examining the polar heating simulation, you might want to plot the polar heating equilibrium profile:

In [ ]:
#makes polar heating profile

teq_polar=np.empty((len(lev),len(lat)))


q0=3
y0=90*np.pi/180
sigy=16*np.pi/180
z0=1
sigz=0.250
for ll in range(len(lev)):
    forc=q0*np.exp(-((lat.values*np.pi/180-y0)**2/(2*sigy**2) + (lev.isel(lev=ll).values/ps-z0)**2/(2*sigz**2)))
    teq_polar[ll,:]=(315-Ty*np.sin(lat.values*np.pi/180)**2-Tz*np.log(lev.isel(lev=ll).values/ps)*\
         np.cos(lat.values*np.pi/180)**2+forc)*(lev.isel(lev=ll).values/ps)**kappa
    
teq_polar[teq_polar<200]=200

#plot the default equilibrium temperature profile
f=plt.figure(figsize=(6,8))
gs=GridSpec(2,1)

fig=plt.subplot(gs[0,0])
plt.contourf(lat,lev,teq_polar,tlevs,extend='both')
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)');
cb=plt.colorbar()
cb.set_label('temperature (K)')
plt.title('polar heating profile')

fig=plt.subplot(gs[1,0])
plt.contourf(lat,lev,teq_polar-teq,tlevs_diff,extend='both',cmap=plt.cm.RdBu_r)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)');
cb=plt.colorbar()
cb.set_label('$\Delta$ temperature (K)')
plt.title('polar heating profile minus default profile')

plt.tight_layout()

Plot the difference of the Control and Experiment temperature with the equilibrium profile. <br>

If using the polar amplification heating, be sure to also examine the polar heating equilbrium profile -- marked in ALL CAPS below where need to change.

In [ ]:
#plot the equilibrium temperature profile difference in Control and Exp


#define the figure and multipanels
f=plt.figure(figsize=(6,7))
gs=GridSpec(2,1)

tlevs_eqdiff=np.arange(-20,20.1,2)


plt.subplot(gs[0,0])
plt.contourf(lat,lev,T_con.mean('lon').mean('time')-teq,tlevs_eqdiff,extend='both',cmap=plt.cm.RdBu_r)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)');
cb=plt.colorbar()
cb.set_label('$\Delta$ temperature (K)')
plt.title('control simulation minus default temperature profile')

###ACTIVITY 2: IN THIS PLOT, IF EXAMINING POLAR HEATING, SWITCH teq WITH teq_polar

plt.subplot(gs[1,0])
plt.contourf(lat,lev,T_exp.mean('lon').mean('time')-teq,tlevs_eqdiff,extend='both',cmap=plt.cm.RdBu_r)
plt.gca().invert_yaxis()
plt.xlabel('latitude')
plt.ylabel('pressure (HPa)');
cb=plt.colorbar()
cb.set_label('$\Delta$ temperature (K)')
plt.title('experiment minus default temperature profile')

plt.tight_layout()

In some of these simulations, the climate response can be understood (to first order and perhaps naively) through an application of the thermal wind relationship in a small layer of the atmosphere (http://paoc.mit.edu/12307/front/thermal%20wind/thermal_wind.pdf):

$\frac{\partial u}{\partial z} = -\frac{g}{f\rho} \frac{\partial T}{\partial y}$

You can also try applying metrics for baroclinicity with a simple relationship for how eddies converge momentum back into the mean flow in the midlatitudes. An easy metric for baroclinicity is the Eady Growth rate: <br>

$\sigma_{Eady}=0.31\frac{g}{N T}|\frac{\partial T}{\partial y}|$

$\frac{\partial T}{\partial y}$ is the meridional temperature gradient, a parameter that some of you may have directly manipulated. If you doubled the meridional temperature gradient in the equilibrium profile, you'd expect to see an increase in the zonal mean jet by the thermal wind relation and by thinking about eddy interactions with the mean flow:

$\frac{\partial [u]}{\partial t} \sim - \frac{\partial [u'v']}{\partial y}$

Increasing the meridional temperature gradient increases the Eady growth rate, which should be related to an increase in the convergence of momentum by eddies ($-\frac{\partial [u'v']}{\partial y}$) and an increase in the zonal mean zonal wind ($[u]$).  

$N$ is the Brunt-Vaisala frequency, which increases with stratification, which some of you may have directly manipulated. Increasing N decreases Eady growth rate. Decreasing baroclinicity means that there are fewer or weaker eddies (midlatitude storms) that form (at that latitude) and that less momentum is converged back into the zonal mean flow (eddy-driven jet) at that latitude. As a result, you should see a weakening of the zonal wind at a latitude where there is increasing stratification.

If the latitudes with largest baroclinicity change location, then the convergence of momentum by eddies into the mean flow should shift locations as well. For example, check out the reasoning in this popular paper by Yin (2005): https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005GL023684

To fully diagnosis the changes in the zonal wind in both the time-mean and its variability, rigorous decomposition is needed that examines at how the eddies and mean flow interact. There is a vast and thorough literature on these topics. The CESM version of the Held-Suarez model provides some of the diagnostics needed to get a start in that direction (u'v', u'u', u'T', etc.).

#### Group discussion questions:
- Where in the extratropics do you see a change in stratification in the experiment (change in temperature with height)? What sign is it?  Focusing on the Northern Hemisphere only will keep the negative signs (from Coriolis parameter) at bay.
- Where do you see a change in the meridional temperature gradient in the experiment? What sign is it? 
- Do you see a decrease or increase in zonal wind speed at that latitude? 
- Based on the changes in baroclinicity (Eady growth rate), does this change in the zonal wind strength make sense?  

### Final thoughts on the Held and Suarez dry dynamical core
- What kind of polar work has been done with the Held-Suarez models? Check out the papers that have cited Held and Suarez (1994) and search the page for words like polar, Arctic, Antarctic, etc: https://journals.ametsoc.org/doi/citedby/10.1175/1520-0477%281994%29075%3C1825%3AAPFTIO%3E2.0.CO%3B2
- What strengths and limitations does the Held/Suarez model have?
- What kinds of parts of the climate is this model best for studying?
 
